# Estrazione Nomi

## Da pdf o txt

In [1]:
import os
import re
import textract
import lib.words as wd

In [9]:
#nomi = list(n.strip().lower() for n in open('../extraction/nomi/nomi_asti.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_gnorry.txt'))
nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_italiani.txt'))
#cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi.txt'))
cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi_big.txt'))

In [3]:
nomi = nomi[:200]
v = [r.split('\t')[0] for r in nomi]

In [5]:
v += ['ambrogio', 'paolina', 'irma']

In [3]:
def extract_notaio_from_doc(dirr, is_pdf=False, max=-1):
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        else: 
            with open (dirr + f, 'r') as f2:
                text = f2.read()
        text = wd.replace_evil_dots_and_underscores(text, rep=r' ').replace('\n',' ').replace(',','')
        text = re.sub('\s+', ' ', text).strip()
        found = False
        for i,line in enumerate(wd.split_sents(text)):
            if found:
                break
            l = line.strip()
            if l != '':
                words = l.split(' ')
                for word in words:
                    if word.lower() == 'me' or word.lower() == 'sottoscritto':
                        index = words.index(word)
                        print f, ' '.join(words[index+1:index+6])
                        found = True
                        break
        if not found:
            print 'NON TROVATO', '\t\t', f

In [62]:
extract_notaio_from_doc('../extraction/files_to_label/', True)

dott LUCIA UCCELLI notaio
dott Ambrogio Romano Notaio
Dottoressa Paolina De Liguori
dottor GIOVANNI MURARO Notaio
dottor VINCENZO CALDERINI Notaio
dottor FABRIZIO PASCUCCI notaio
Alessandra Bianchi notaio in
Avv Lucia Di Lieto
Dottor Roberto Colucci Notaio
dottor DONATO CASCAVILLA Notaio
dottor Maria Luisa Sperandeo
Andrea Guglielmoni Notaio in
dottor TOMMASO GAETA Notaio
PAOLA TOTARO Notaio in
Giovanna Falco notaio in
dottor Michele Colasanto Notaio
dottor FABRIZIO PASCUCCI Notaio
Avv Giovanni Piacitelli Notaio
Dottor Notaio PAOLO MICHELI
Dottor Marco Cavallini Notaio


In [39]:
extract_notaio_from_doc('../atti_costitutivi_txt/', False, 500)

5049618730001.txt Fabrizio Figurelli Notaio in Cavezzo
4930258720001.txt Dr Raimondo MALINCONICO Notaio in
5003601230001.txt Dott Andrea Coletta Notaio in
5060167590001.txt Dottor LUCIANO QUAGGIA Notaio in
4985610910001.txt dott PIETRO BERNARDI FABBRANI Notaio
4934063060001.txt dottor FILIPPO MARINELLI Notaio in
4943577600001.txt Zaira LO BIANCO Notaio in
5024081290001.txt Dottor ANGELO CACCETTA Notaio in
5005776590001.txt FABIO MONACO notaio in Viareggio
5051922110001.txt dottor Francesco Figlioli notaio residente
4919128160001.txt dottor Giovannibattista Musto Notaio in
5067992100001.txt Dott ssa ORFINA SCROCCO Notaio
5002413830001.txt presso il Dottor MARCO Collegio
5044504790001.txt dott ssa ANNA LAURA PAOLETTI
5052464430001.txt dottor Luigi BARTOLOMEO Notaio in
5121967110001.txt dott Roberto Chiari fu Leopoldo
5090226530001.txt dottor Salvatore Cutrupia notaio in
4923509370001.txt dr Pasquale Salvatore Placida Notaio
4985596990001.txt dottor GIOVANNI MURARO Notaio del
511074992000

## Dalle frasi

In [4]:
def extract_notaio_from_sentences(doc):
    '''
    Estrai notaio da un documento.
    Arguments:
        - doc: il documento (lista di frasi)
    Returns:
        - lista di nomi, lista di cognomi
    '''
    
    whitelist = ['de', 'di'] #, 'dei', 'del', 'della', 'degli', 'delle', 'da', 'dello']
    
    for frase in doc:
        frase = frase.replace('\n',' ').replace(',','')
        frase = re.sub('\s+', ' ', frase).strip()
        if frase != '':
            words = frase.split(' ')
            for word in words:
                if word.lower() == 'me' or word.lower() == 'sottoscritto' or word.lower() == 'mir':
                    index = words.index(word)
                    sub = words[index+1:index+6]
                    res = ''
                    for w in sub:
                        if w.lower() in nomi or w.lower() in cognomi or w.lower() in whitelist:
                            res += w + ' '
                    return res     
                    
    return ""

In [82]:
for i,f in enumerate(os.listdir('../atti_costitutivi_txt/')):
    if i == 500:
        break
    text = ""    
    with open ('../atti_costitutivi_txt/' + f, 'r') as f2:
        text = f2.read()
    sents = wd.sentences_doc(text, rep=r' ')    
    res = extract_notaio_from_sentences(sents)
    print i, f, '\t', res    

0 5049618730001.txt 	Fabrizio Cavezzo 
1 4930258720001.txt 	Raimondo MALINCONICO 
2 5003601230001.txt 	Andrea Coletta 
3 5060167590001.txt 	LUCIANO 
4 4985610910001.txt 	PIETRO BERNARDI 
5 4934063060001.txt 	FILIPPO MARINELLI 
6 4943577600001.txt 	Zaira BIANCO 
7 5024081290001.txt 	ANGELO 
8 5005776590001.txt 	FABIO MONACO 
9 5051922110001.txt 	Francesco 
10 4919128160001.txt 	Giovannibattista Musto 
11 5067992100001.txt 	ORFINA 
12 5002413830001.txt 	MARCO 
13 5044504790001.txt 	ANNA LAURA PAOLETTI 
14 5052464430001.txt 	Luigi BARTOLOMEO 
15 5121967110001.txt 	Roberto Chiari Leopoldo 
16 5090226530001.txt 	Salvatore Cutrupia 
17 4923509370001.txt 	Pasquale Salvatore Placida 
18 4985596990001.txt 	GIOVANNI MURARO 
19 5110749920001.txt 	Gianmario ANGELINO 
20 4928305720001.txt 	Gigino ROLLO 
21 4954895450001.txt 	Gianluca ANDERLINI 
22 4914089470001.txt 	MICHELE BIANCHI 
23 5082973800001.txt 	PAOLO 
24 5058490980001.txt 	VITTORIO GIUA MARASSI 
25 4933630010001.txt 	di FRANCESCO 
26 4909

##### Ignora

In [38]:
def extract_notaio_from_end(doc):
    whitelist= ['notaio', '(l.s.)', '(segue sigillo)', 'impronta']
    puntegg = ['\n', '.', ',', ';']
    for frase in reversed(doc):
        if frase != '':
            words = wd.splitted_words_utf8(frase)
            for word in words:
                if word.lower() in whitelist:
                    index = words.index(word)
                    begin = -1
                    print f, '\t', ' '.join(words[:index])
                    for w in reversed(words[:index]):
                        if any(p in w for p in puntegg): 
                            begin = words.index(w)
                    if begin > -1:
                        print f, '\t', ' '.join(words[begin:index])
                        return
                    return
   
    return "NOPE"

In [39]:
for i,f in enumerate(os.listdir('../atti_costitutivi_txt/')):
    if i == 10:
        break
    text = ""    
    with open ('../atti_costitutivi_txt/' + f, 'r') as f2:
        text = f2.read()
    sents = wd.sentences_doc(text, rep=r' ')  
    extract_notaio_from_end(sents)
    #print i, f, '\t', res   

5049618730001.txt 	F to Fabrizio Figurelli
4930258720001.txt 	Salerno 16 febbraio 2017 Firmato digitalmente Raimondo Malinconico
5003601230001.txt 	Firmato Dosio Roberta Pallocchini Stefano Usini Giorgio Andrea Coletta
5060167590001.txt 	Firmato Jorillo Hans Firmato Gilardi Roberto Firmato Luciano Quaggia
4985610910001.txt 	F to Pietro Bernardi Fabbrani
4934063060001.txt 	F to VERBICARO Ruben VERBICARO Noemi Sergio CALVANO Filippo MARINELLI
4943577600001.txt 	Firmato Cicala Pasqualino Al Refaie Fares Restuccia Mariaelisa Zaira Lo Bianco
5024081290001.txt 	Firmato Auteri Mamta Angelo Caccetta
5005776590001.txt 	F to Roberta Girolami F to Vinicio Tuccori F to Fabio Monaco
5051922110001.txt 	Bollo assolto ai sensi del Decreto 22 febbraio 2007 mediante M U I Atto registrato telematicamente presso l Agenzia delle Entrate Ufficio Territo riale di Palermo 1 in data 05 maggio 2017 al n 5201 serie 1T Palermo nel mio studio Via Principe di Paternò n 42 Lì diciassette maggio duemiladiciassette Fi

# Estrazione STATUTO

In [33]:
def extract_statuto_from_sentences_1(dirr, is_pdf=False, max=-1):
    begin_statuto = ['STATUTO','FUNZIONAMENTODELLASOCIET']
    text = ""
    inizio = -1
    index = -1
    fine = -1
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    
        notaio = extract_notaio_from_sentences(sents)  
        print f
        print 'notaio', notaio, '\n'

        for frase in sents:  
            if any(b in re.sub('\s+', '', frase) for b in begin_statuto):
                #print i, f, '\t', frase, '\n'
                inizio = sents.index(frase)
            if notaio.lower() in frase.lower() and inizio > 0:
                fine = sents.index(frase)
                break
            
        if fine > -1:
            #print sents[inizio][index:]+'\n'+'\n'.join(sents[inizio+1:fine+1]) + '\n\n\n'
            print '\n'.join(sents[inizio:fine+1]) + '\n\n\n'

In [55]:
def extract_statuto(dirr, is_pdf=False, max=-1):
    begin_statuto = ['STATUTO','FUNZIONAMENTODELLASOCIET']
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    

        for frase in sents:  
            if any(b in re.sub('\s+', '', frase) for b in begin_statuto):
                print i, f, '\t', frase, '\n'

In [12]:
def extract_statuto_from_sentences(dirr, is_pdf=False, max=-1):
    begin_statuto = ['S T A T U T O', 'STATUTO', 'STATUTO DELLA SOCITET', 'FUNZIONAMENTO DELLA SOCIET', 'PATTI DISCIPLINANTI']
    
    for i,f in enumerate(os.listdir(dirr)):
        text = ""
        inizio = -1
        index = -1
        fine = -1
        if i == 0:
            continue
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        sents = wd.sentences_doc(text, rep=r' ')    
        notaio = extract_notaio_from_sentences(sents)  
        print f
        print 'notaio', notaio, '\n'
        for k, frase in enumerate(sents):
            if index == -1:
                for b in begin_statuto:
                    j = wd.index_ignore_whitespaces(frase, b)
                    if j > -1:
                        inizio = k
                        index = j
                        break
                
            if notaio.lower() in frase.lower() and inizio > 0:
                fine = sents.index(frase)
                if fine > inizio:
                    break
            
        if fine > -1:
            print sents[inizio][index:]+'\n'+'\n'.join(sents[inizio+1:fine+1]) + '\n\n\n'

In [13]:
extract_statuto_from_sentences('../extraction/files_to_label/', True, 2)

4907876150001.pdf
notaio Ambrogio Romano  

FUNZIONAMENTO DELLA SOCIETA' "CAFFETTERIA 74 Società cooperativa sociale onlus" ART1  DENOMINAZIONE SOCIALE 1 1 E' costituita, in conformità al disposto degli artt 2511 e segg  cod civ  e nel rispetto dei dettami di cui alla Legge 8 novembre 1991, n 381, e successive modifiche ed integrazioni, una società cooperativa sociale a mutualità prevalente denominata "CAFFETTERIA 74 Società cooperativa sociale onlus"
 1 2 La società, la quale si propone di agire con i terzi ed è disciplinata, in conformità al disposto dell'art 2519 cod civ , dalle norme dettate in tema di società a responsabilità limitata in quanto compatibili, dovrà assolvere all'obbligo di indicare negli atti e nella corrispondenza il numero di iscrizione presso l'albo delle cooperative a mutualità prevalente
 1 3 La società potrà, nel rispetto della normativa vigente, essere parte di gruppi cooperativi paritetici, nonchè, con deliberazione dei soci, aderire ad associazioni nazional